Load libraries

In [37]:
library(tensorflow)
library(EBImage)

Make dataframe with files and labels

In [38]:
files = list.files('db', recursive = TRUE, full.names= TRUE)

labels= lapply(strsplit(files, '[/]'), function(x){
    x[[2]]
    
})
labels = as.numeric(labels)

data = data.frame('file' = files, 'label'= labels)

Split images in train and test


In [39]:
split = sample(x =  c(1:nrow(data)), size = round(0.8*nrow(data)) )
train = data[split,]
test = data[-split,]          

Set parameters

In [52]:
aantal_pool  = 2
aantal_kanalen = 64
clas = as.integer(62)#number of classes
schaal = 1 #scale of pixel values

out = 0.5 #dropout
batch_train = 50 #batchsize
batch_test = 200 #batchsize test
ds = 0.999 #gradient descent
lr = 1e-4 #learningrate

h = as.integer(60) #heigth image
w = as.integer(60) #width image
kanalen = as.integer(3) #chanals of image

Place holders

In [41]:
x <- tf$placeholder(tf$float32, shape(NULL, h,w,kanalen), 'x')
#target values
labels <- tf$placeholder(tf$int64, shape(NULL), 'labels')
#dropout rate
keep_prob <- tf$placeholder(tf$float32, shape(),'keep_prob')
#learningrate
lrate <- tf$placeholder(tf$float32, shape(), 'lrate')

Define variables

In [42]:
w_conv1 <-  tf$Variable( tf$truncated_normal(shape(5L, 5L, 3L, 40L),stddev=0.1), 'w_conv1')
b_conv1 <- tf$Variable( tf$truncated_normal( shape(40L),stddev=0.1), 'b_conv1')

w_conv2 <- tf$Variable( tf$truncated_normal(shape = shape(5L, 5L, 40L, 64L), stddev=0.1), 'w_conv2')
b_conv2 <- tf$Variable( tf$truncated_normal(shape = shape(64L), stddev=0.1), 'b_conv2')

w_conv3 <- tf$Variable( tf$truncated_normal(shape = shape(5L, 5L, 64L, 64L), stddev=0.1), 'w_conv3')
b_conv3 <- tf$Variable( tf$truncated_normal(shape = shape(64L), stddev=0.1), 'b_conv3')

w_fc1 <- tf$Variable( tf$truncated_normal(shape((w*h)/(4^(aantal_pool)) * aantal_kanalen, 1024L), stddev=0.1), 'w_fc1')
b_fc1 <- tf$Variable( tf$truncated_normal(shape(1024L), stddev=0.1), 'b_fc1')

w_output <- tf$Variable( tf$truncated_normal(shape(1024L, clas), stddev=0.1), 'w_output')
b_output <- tf$Variable( tf$truncated_normal(shape(clas), stddev=0.1), 'b_output')

Define network

In [43]:
h_conv1 <- tf$nn$relu( tf$nn$conv2d(x, w_conv1 , strides=c(1L, 1L, 1L, 1L), padding='SAME') + b_conv1)
h_pool1 <- tf$nn$max_pool(h_conv1, ksize=c(1L, 2L, 2L, 1L),strides=c(1L, 2L, 2L, 1L), padding='SAME')
h_conv2 <- tf$nn$relu( tf$nn$conv2d(h_pool1, w_conv2, strides=c(1L, 1L, 1L, 1L), padding='SAME') + b_conv2)
h_pool2 <- tf$nn$max_pool(h_conv2,  ksize=c(1L, 2L, 2L, 1L),strides=c(1L, 2L, 2L, 1L), padding='SAME')
h_conv3 <- tf$nn$relu( tf$nn$conv2d(h_pool2, w_conv3, strides=c(1L, 1L, 1L, 1L), padding='SAME') + b_conv3)
h_conv3_flat <- tf$reshape(h_conv3, shape(-1L, (w*h)/(4^(aantal_pool)) * aantal_kanalen))
h_fc1 <- tf$nn$relu(tf$matmul(h_conv3_flat, w_fc1) + b_fc1)
h_fc1 <- tf$nn$relu(tf$matmul(h_conv3_flat, w_fc1) + b_fc1)
h_fc1_drop <- tf$nn$dropout(h_fc1, keep_prob)
h_output <- tf$nn$softmax(tf$matmul(h_fc1_drop, w_output) + b_output)

Define cost function

In [44]:
cost = tf$nn$sigmoid_cross_entropy_with_logits( logits = h_output, labels = tf$one_hot(labels, clas))

Define optimizer

In [45]:
train_step <- tf$train$AdamOptimizer(lrate)$minimize(cost)

Define some variables to print along the way

In [46]:
correct_prediction <- tf$equal(tf$argmax(h_output, 1L),labels)
accuracy <- tf$reduce_mean(tf$cast(correct_prediction, tf$float32))

Define the session

In [47]:
sess <- tf$InteractiveSession()
sess$run(tf$global_variables_initializer())

Train the network

In [48]:
read_images = function(files){
    batch = array(0, dim = c(length(files), w, h, kanalen))
for(i in 1:length(files)){
    file = batch_files[i]
    im = readImage( as.character(file))
    im = resize(im, w=w, h=h)
    
    batch[i,,,] = im
}
    return(batch)
}

In [60]:
for (i in 1:20000) {
  
  #lees 50 random plaatjes in
 samp = sample( x=  c(1: nrow(train)) , size = batch_train )
  
  batch_labels = as.vector(train$label[samp])
  batch_files = train$file[samp]

batch_files = read_images(batch_files)
  
  #train met gradient descent
  sess$run(train_step, feed_dict = dict(x = batch_files , labels = batch_labels , keep_prob = out, lrate = ds^i*lr))
  
  
  
  
  
  #valideer om de 100 keer hoe het gaat op de testset
  if (i %% 100 == 0) {
    
  
    #evalueer op de testset
    samp = sample( x=  c(1: nrow(test)) , size = batch_test )
  
  batch_labels = as.vector(test$label[samp])
  batch_files = test$file[samp]

batch_files = read_images(batch_files)
  
  #train met gradient descent
 test_accuracy =  sess$run(accuracy, feed_dict = dict(x = batch_files , labels = batch_labels , keep_prob = 1))
  
  
    print( paste("step:", i, "test accuracy:", test_accuracy) )    
  }
  
  

  
  
  

}


ERROR: Error in py_call_impl(callable, dots$args, dots$keywords): TypeError: 'NoneType' object is not callable

Detailed traceback: 
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/client/session.py", line 889, in run
    run_metadata_ptr)
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/client/session.py", line 1120, in _run
    feed_dict_tensor, options, run_metadata)
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/client/session.py", line 1317, in _do_run
    options, run_metadata)
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/client/session.py", line 1323, in _do_call
    return fn(*args)
  File "/usr/local/lib/python2.7/dist-packages/tensorflow/python/client/session.py", line 1302, in _run_fn
    status, run_metadata)



This network sould be able to reach an accuracy of around 98 procent. If you do some data augmentation you can get over 99 percent.